# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [2]:
import pandas as pd
import requests
import numpy as np

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [3]:
# 1) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'

requests.get(url)
r = requests.get(url)
r.status_code

r.json()
data_json = r.json()
# print(data_json['items'])

df = pd.DataFrame(data_json['items'])
df.head(3)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,9234,Município,Nova Friburgo,3303401,RJ,None,Deferido,17944.000933/2013-72,2013-08-30T03:00:00Z,Operação contratual interna,Provias,Instituição Financeira Nacional,Banco do Brasil S/A,Real,14000000.00,1,0,09/10/2013
1,9317,Município,Nova Iguaçu,3303500,RJ,None,Regularizado,17944.000953/2007-03,2007-06-18T03:00:00Z,Operação contratual interna,Regularização de Dívida - Outros,Outras Organizações,Pessoa Jurídica,Real,1077466.66,1,0,02/08/2007
2,10412,Município,Maricá,3302700,RJ,None,Arquivado por decurso de prazo,17944.001193/2011-20,2012-09-12T03:00:00Z,Operação contratual interna,PAC 2 - Pró-Transporte - Pavimentação e Qualif...,Instituição Financeira Nacional,Caixa Econômica Federal,Real,22800000.00,0,0,17/10/2012


In [4]:
# 2) Seu código aqui
df['status'].value_counts()
# ou 
df.status.value_counts()
# visualizar apenas os 3 primeiros
df.status.value_counts().head(3)

Deferido        96
Arquivado       53
Regularizado    26
Name: status, dtype: int64

In [5]:
# 3) Seu código aqui
df['data_status'] = df['data_status'].astype('string')
df.dtypes
df['ano'] = df['data_status'].str[6:]
df['ano']

0      2013
1      2007
2      2012
3      2007
4      2011
       ... 
237    2011
238    2010
239    2022
240    2019
241    2019
Name: ano, Length: 242, dtype: string

In [6]:
df.head(1)
# resposta faltante frequencia com que os anos aparecem na base de dados
df['ano'].value_counts()

2008    37
2007    29
2014    20
2010    19
2019    17
2013    15
2020    15
2012    13
2023    11
2022    11
2011    10
2009     9
2015     8
2021     5
2017     5
2016     4
2002     4
2018     4
2006     4
2004     1
2003     1
Name: ano, dtype: Int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [11]:
#1) Seu código aqui
def modulo4(uf:str,busca:str):
    if busca in ('estado', 'Estado'):
        busca = 'Estado'
    elif busca in ('municipio', 'município'):
        busca = 'Munic%C3%ADpio'
    url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf='+uf+'&tipo_interessado='+busca
    r = requests.get(url)
    resposta = r.json()
    return pd.DataFrame(resposta['items'])

modulo4('RJ','municipio')

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,9234,Município,Nova Friburgo,3303401,RJ,None,Deferido,17944.000933/2013-72,2013-08-30T03:00:00Z,Operação contratual interna,Provias,Instituição Financeira Nacional,Banco do Brasil S/A,Real,14000000.00,1,0,09/10/2013
1,9317,Município,Nova Iguaçu,3303500,RJ,None,Regularizado,17944.000953/2007-03,2007-06-18T03:00:00Z,Operação contratual interna,Regularização de Dívida - Outros,Outras Organizações,Pessoa Jurídica,Real,1077466.66,1,0,02/08/2007
2,10412,Município,Maricá,3302700,RJ,None,Arquivado por decurso de prazo,17944.001193/2011-20,2012-09-12T03:00:00Z,Operação contratual interna,PAC 2 - Pró-Transporte - Pavimentação e Qualif...,Instituição Financeira Nacional,Caixa Econômica Federal,Real,22800000.00,0,0,17/10/2012
3,13579,Município,Rio de Janeiro,3304557,RJ,None,Arquivado,19405.000029/2006-61,2006-07-20T03:00:00Z,Operação contratual interna,Saneamento básico,Instituição Financeira Nacional,Caixa Econômica Federal,Real,1316493.93,0,0,26/06/2007
4,6363,Município,Rio Claro,3304409,RJ,None,Regularizado,17944.000328/2008-34,2011-06-01T03:00:00Z,Operação contratual interna,Regularização de Dívida - Energia Elétrica,Empresa Não Estatal,Light - Serviços de Eletricidade S/A,Real,6733230.99,0,0,22/07/2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,11053,Município,Macaé,3302403,RJ,None,Deferido,17944.001346/2011-39,2011-11-29T02:00:00Z,Operação contratual interna,PAC 2 - Pró-Moradia,Instituição Financeira Nacional,Caixa Econômica Federal,Real,48286602.68,1,0,08/12/2011
238,5643,Município,São João de Meriti,3305109,RJ,None,Arquivado,17944.000161/2010-26,2010-10-15T03:00:00Z,Operação contratual interna,PAC - Saneamento - 2009 A,Instituição Financeira Nacional,Caixa Econômica Federal,Real,50074939.06,0,0,20/12/2010
239,56075,Município,Resende,3304201,RJ,PVL02.003504/2022-85,Deferido (PVL-IF),None,None,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Banco do Brasil S/A,Real,2160000.00,0,1,28/07/2022
240,34099,Município,Resende,3304201,RJ,PVL02.003090/2019-99,Deferido (PVL-IF),None,None,Operação contratual interna,Aporte de capital,Instituição Financeira Nacional,Caixa Econômica Federal,Real,4500000.00,0,1,27/09/2019


In [8]:
# 2) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=MG&tipo_interessado=Estado'
r = requests.get(url)
resposta = r.json()
df = pd.DataFrame(resposta['items'])

filtro = df.loc[df['status'] == 'Arquivado por decurso de prazo']
filtro.status.value_counts()

Arquivado por decurso de prazo    1
Name: status, dtype: int64

In [9]:
# 3) Seu código aqui
r = requests.get('https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=BA&tipo_interessado=Munic%C3%ADpio')
resposta = r.json()
df = pd.DataFrame(resposta['items'])

deferido = df[df['status'] == 'Deferido']
deferido[['interessado', 'status']].value_counts()
#df.groupby('interessado')['status'].count()
#df[['status', 'interessado']].value_counts()


interessado             status  
Luís Eduardo Magalhães  Deferido    15
Vitória da Conquista    Deferido    11
Camaçari                Deferido    10
Barreiras               Deferido    10
Lauro de Freitas        Deferido    10
                                    ..
Malhada                 Deferido     1
Licínio de Almeida      Deferido     1
Lençóis                 Deferido     1
Cardeal da Silva        Deferido     1
Xique-Xique             Deferido     1
Length: 179, dtype: int64

In [10]:
# 4) Seu código aqui
r = requests.get('https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=BA&tipo_interessado=Estado')
resposta = r.json()

df = pd.DataFrame(resposta['items'])
df.to_csv('baseBahia.csv')